<a href="https://colab.research.google.com/github/nptan2005/spark401_colab/blob/main/notebooks/Spark_lab_03_04_05_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y openjdk-17-jdk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jre session-migration x11-utils
Suggested packages:
  gvfs libxt-doc openjdk-17-demo openjdk-17-source visualvm mesa-utils
The following NEW packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jdk openjdk-17-jr

In [2]:
!wget https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
!tar xf spark-4.0.1-bin-hadoop3.tgz

--2026-01-10 06:28:30--  https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548955321 (524M) [application/x-gzip]
Saving to: ‘spark-4.0.1-bin-hadoop3.tgz’

spark-4.0.1-bin-had 100%[===================>] 523.52M  12.5MB/s    in 66s     

2026-01-10 06:29:37 (7.91 MB/s) - ‘spark-4.0.1-bin-hadoop3.tgz’ saved [548955321/548955321]



In [3]:
# ===============================
# Spark 4.0.1 Setup (REQUIRED)
# ===============================
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-4.0.1-bin-hadoop3"
os.environ["PATH"] += ":/content/spark-4.0.1-bin-hadoop3/bin"

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("lab").getOrCreate()

print("Spark version:", spark.version)

Spark version: 4.0.1


# 🟢 LAB 3 – FILTER & IO (ĐỌC ÍT HƠN = NHANH HƠN)

## 🎯 Mục tiêu LAB 3
*	Hiểu Spark KHÔNG đọc hết dữ liệu
*	Hiểu Predicate Pushdown
*	**Phân biệt:**
-	filter() trước / sau khi đọc
-	Spark vs Pandas
-	Biết viết Spark job “đỡ tốn IO”

---

## 0️⃣ Nhắc lại tư duy sống còn

* ❌ Spark không nhanh vì nhiều core
* ✅ Spark nhanh vì đọc ÍT dữ liệu hơn

---

## 1️⃣ Tạo dữ liệu giả lập (có điều kiện lọc)

```python
from pyspark.sql import functions as F

df = (
    spark.range(0, 5_000_000)
    .withColumn("country", F.when(F.col("id") % 5 == 0, "VN")
                            .when(F.col("id") % 5 == 1, "SG")
                            .when(F.col("id") % 5 == 2, "TH")
                            .when(F.col("id") % 5 == 3, "ID")
                            .otherwise("MY"))
    .withColumn("amount", (F.rand() * 10000).cast("double"))
)

df.count()
```

---

## 2️⃣ Ghi dữ liệu ra Parquet (giả lập Data Lake)

```python
df.write.mode("overwrite").parquet("/tmp/lab3_orders")
```

📌 Parquet = columnar format → điều kiện để Spark pushdown

---

## 3️⃣ Case 1 – Filter SAU khi đọc (cách NGU)

```python
df_read = spark.read.parquet("/tmp/lab3_orders")

df_bad = df_read.filter(F.col("country") == "VN")

df_bad.explain()
```

### ❌ Điều gì xảy ra?
*	Spark đọc toàn bộ file
*	Sau đó mới filter
*	IO lớn → chậm → tốn tiền

---

## 4️⃣ Case 2 – Filter NGAY KHI đọc (chuẩn)

```python
df_good = (
    spark.read.parquet("/tmp/lab3_orders")
    .filter(F.col("country") == "VN")
)

df_good.explain()
```

### 🔍 Nhìn trong explain():

#### Bạn sẽ thấy:

```code
PushedFilters: [IsNotNull(country), EqualTo(country,VN)]
```

🎉 ĐÂY CHÍNH LÀ PREDICATE PUSHDOWN

---

## 5️⃣ Predicate Pushdown là gì? (Hiểu cho đúng)

Spark đẩy điều kiện lọc xuống tận file storage

### 📦 Parquet file:
*	Có metadata theo column
*	Spark chỉ đọc block cần thiết

> 📌 IO giảm mạnh

---

## 6️⃣ Spark vs Pandas – khác biệt sống còn

### Pandas

```python
df = pd.read_parquet("big.parquet")
df[df["country"] == "VN"]
```

❌ Đọc toàn bộ file vào RAM

---

### Spark

```python
spark.read.parquet("big.parquet").filter(col("country") == "VN")
```

✅ Chỉ đọc phần cần

---

## 7️⃣ Những filter KHÔNG pushdown được (rất hay sai)

### ❌ UDF

```python
df.filter(my_udf(col("country")))
```

### ❌ Function phức tạp

```python
df.filter(F.lower(F.col("country")) == "vn")
```

### ❌ Python logic

```python
df.filter(lambda x: x.country == "VN")
```

👉 Spark buộc phải đọc hết

---

## 8️⃣ Filter đúng chuẩn bank-grade

```python
df.filter(
    (F.col("country") == "VN") &
    (F.col("amount") > 1000)
)
```

### 📌 Điều kiện:
*	Cột gốc
*	Không UDF
*	Không transform

⸻

## 9️⃣ Partition + Filter = combo mạnh

```python
df_partitioned = (
    df.write
    .partitionBy("country")
    .mode("overwrite")
    .parquet("/tmp/lab3_partitioned")
)

spark.read.parquet("/tmp/lab3_partitioned") \
    .filter(F.col("country") == "VN") \
    .explain()
```

🎯 Spark chỉ đọc folder country=VN

---

## 🔟 Tổng kết LAB 3

### ❌ Sai lầm phổ biến
*	Đọc rồi mới filter
*	Dùng UDF để filter
*	Nghĩ Spark nhanh vì nhiều core

### ✅ Đúng chuẩn
*	Filter càng sớm càng tốt
*	Dùng column gốc
*	Parquet + predicate pushdown
*	Partition theo business key

---

## 🧠 CÂU HỎI BẮT BUỘC (TRƯỚC LAB 4)
1.	Vì sao filter sau khi read là tốn IO?
2.	Predicate pushdown hoạt động ở tầng nào?
3.	Vì sao UDF phá predicate pushdown?
4.	PartitionBy giúp gì cho filter?
5.	Khi nào KHÔNG nên partitionBy?


In [4]:
from pyspark.sql import functions as F

df = (
    spark.range(0, 5_000_000)
    .withColumn("country", F.when(F.col("id") % 5 == 0, "VN")
                            .when(F.col("id") % 5 == 1, "SG")
                            .when(F.col("id") % 5 == 2, "TH")
                            .when(F.col("id") % 5 == 3, "ID")
                            .otherwise("MY"))
    .withColumn("amount", (F.rand() * 10000).cast("double"))
)

In [5]:
df.count()

5000000

In [5]:
df.write.mode("overwrite").parquet("/content/tmp/lab3_orders")

In [8]:
# ton IO

df_read = spark.read.parquet("/content/tmp/lab3_orders")

df_bad = df_read.filter(F.col("country") == "VN")

df_bad.explain()

== Physical Plan ==
*(1) Filter (isnotnull(country#11) AND (country#11 = VN))
+- *(1) ColumnarToRow
   +- FileScan parquet [id#10L,country#11,amount#12] Batched: true, DataFilters: [isnotnull(country#11), (country#11 = VN)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/tmp/lab3_orders], PartitionFilters: [], PushedFilters: [IsNotNull(country), EqualTo(country,VN)], ReadSchema: struct<id:bigint,country:string,amount:double>




In [9]:
# chuan
df_good = (
    spark.read.parquet("/content/tmp/lab3_orders")
    .filter(F.col("country") == "VN")
)

df_good.explain()

== Physical Plan ==
*(1) Filter (isnotnull(country#15) AND (country#15 = VN))
+- *(1) ColumnarToRow
   +- FileScan parquet [id#14L,country#15,amount#16] Batched: true, DataFilters: [isnotnull(country#15), (country#15 = VN)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/tmp/lab3_orders], PartitionFilters: [], PushedFilters: [IsNotNull(country), EqualTo(country,VN)], ReadSchema: struct<id:bigint,country:string,amount:double>




In [11]:
df.filter(
    (F.col("country") == "VN") &
    (F.col("amount") > 1000)
)

DataFrame[id: bigint, country: string, amount: double]

In [6]:
df_partitioned = (
    df.write
    .partitionBy("country")
    .mode("overwrite")
    .parquet("/content/tmp/lab3_partitioned")
)

In [13]:
spark.read.parquet("/content/tmp/lab3_partitioned") \
    .filter(F.col("country") == "VN") \
    .explain()

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [id#18L,amount#19,country#20] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/tmp/lab3_partitioned], PartitionFilters: [isnotnull(country#20), (country#20 = VN)], PushedFilters: [], ReadSchema: struct<id:bigint,amount:double>




# 🔴 LAB 4 – GROUP BY & SHUFFLE (HIỂU = SỐNG)

## 🎯 Mục tiêu LAB 4

*	Hiểu Shuffle là gì (thực sự)
*	Vì sao groupBy() luôn nguy hiểm
*	Biết đọc explain() để nhìn thấy tiền đang cháy
*	Biết viết aggregation bank-grade

---

# 0️⃣ Ôn lại 1 câu sống còn

* ❌ Spark chậm không phải do CPU
* ❌ Spark chết vì SHUFFLE

---

## 1️⃣ Chuẩn bị dữ liệu (dùng lại LAB 3)

```python
from pyspark.sql import functions as F

df = spark.read.parquet("/tmp/lab3_orders")
df.count()
```

---

## 2️⃣ GROUP BY đơn giản (nhưng nguy hiểm)


```pyhon
df_group = (
    df.groupBy("country")
      .agg(F.sum("amount").alias("total_amount"))
)

df_group.explain()
```

### 🔥 Quan sát explain()

Bạn sẽ thấy dòng rất quan trọng:

```code
Exchange hashpartitioning(country, 200)
```

👉 Exchange = SHUFFLE

---

## 3️⃣ Shuffle là gì? (hiểu cho đúng)

**Khi groupBy(country):**

1.	Spark phải gom tất cả record cùng country
2.	Record đang nằm rải rác ở nhiều executor
3.	→ Phải network transfer
4.	→ Disk spill
5.	→ Task imbalance

> 📌 Shuffle = network + disk + sync

---

4️⃣ Vì sao GROUP BY luôn shuffle?

|**Operation**|**Shuffle**|
|-------------|-----------|
|select	|❌|
|filter|	❌|
|withColumn	|❌|
|groupBy|	✅|
|join	|✅|
|orderBy|	✅|

👉 Vì cần data cùng key ở cùng task

---

## 5️⃣ Nhìn shuffle bằng Spark UI (tư duy)

**Trong Spark UI → Stage:**

*	Shuffle Read (MB / GB)
*	Shuffle Write
*	Task time lệch nhau

📌 Stage stuck 99% = skew + shuffle

---

## 6️⃣ Thử GROUP BY trên skew data (thực chiến)

```pyhon
df_skew = (
    df.withColumn(
        "customer_id",
        F.when(F.col("country") == "VN", "HOT")
         .otherwise(F.col("country"))
    )
)

df_skew.groupBy("customer_id").count().explain()
```

### 🔥 Kết quả:

*	1 key = HOT
*	1 task xử lý khổng lồ
*	Các task khác rảnh

⸻

## 7️⃣ Giảm tác hại shuffle – cách 1: pre-aggregation

### ❌ Sai:

```pyhon
df.groupBy("country").sum("amount")
```

### ✅ Đúng (khi có bước trung gian):

```python
df.repartition("country") \
  .groupBy("country") \
  .sum("amount")
```

📌 Không loại shuffle, nhưng giảm cost

---

## 8️⃣ Giảm tác hại shuffle – cách 2: giảm partitions

```pyhon
spark.conf.set("spark.sql.shuffle.partitions", 50)
```

📌 Mặc định = 200

Dataset nhỏ → 200 task = overhead

---

## 9️⃣ Giảm tác hại shuffle – cách 3: AQE (Spark 3+)

```python
spark.conf.set("spark.sql.adaptive.enabled", "true")
```

#### AQE làm gì?
*	Giảm shuffle partitions runtime
*	Detect skew
*	Rebalance tasks

📌 Không phải phép màu – design vẫn quan trọng

---

## 🔟 GROUP BY chuẩn bank-grade

### ❌ Không làm
*	groupBy trên key skew
*	groupBy nhiều cột lớn
*	groupBy trong Gold layer

### ✅ Nên làm
*	Aggregate sớm (Silver)
*	Key ít cardinality
*	Có explain lưu lại
*	Shuffle predictable

⸻

## 🧠 CÂU HỎI BẮT BUỘC – LAB 4

**Trả lời bằng ý hiểu của bạn:**

1.	Vì sao groupBy() luôn gây shuffle?
2.	Exchange trong plan nghĩa là gì?
3.	Vì sao shuffle partitions = 200 nguy hiểm?
4.	Khi nào shuffle là bắt buộc?
5.	Dấu hiệu job đang chết vì shuffle?


In [7]:
from pyspark.sql import functions as F

df = spark.read.parquet("/content/tmp/lab3_orders")
df.count()

5000000

In [8]:
# GROUP BY đơn giản (nhưng nguy hiểm)

df_group = (
    df.groupBy("country")
      .agg(F.sum("amount").alias("total_amount"))
)

df_group.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[country#4], functions=[sum(amount#5)])
   +- Exchange hashpartitioning(country#4, 200), ENSURE_REQUIREMENTS, [plan_id=89]
      +- HashAggregate(keys=[country#4], functions=[partial_sum(amount#5)])
         +- FileScan parquet [country#4,amount#5] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/tmp/lab3_orders], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<country:string,amount:double>




In [9]:
# Thử GROUP BY trên skew data (thực chiến)
df_skew = (
    df.withColumn(
        "customer_id",
        F.when(F.col("country") == "VN", "HOT")
         .otherwise(F.col("country"))
    )
)

df_skew.groupBy("customer_id").count().explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[customer_id#21], functions=[count(1)])
   +- Exchange hashpartitioning(customer_id#21, 200), ENSURE_REQUIREMENTS, [plan_id=104]
      +- HashAggregate(keys=[customer_id#21], functions=[partial_count(1)])
         +- Project [CASE WHEN (country#4 = VN) THEN HOT ELSE country#4 END AS customer_id#21]
            +- FileScan parquet [country#4] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/tmp/lab3_orders], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<country:string>




In [10]:
#  repartition

df_p = df.repartition("country") \
  .groupBy("country") \
  .sum("amount").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[country#4], functions=[sum(amount#5)])
   +- HashAggregate(keys=[country#4], functions=[partial_sum(amount#5)])
      +- Exchange hashpartitioning(country#4, 200), REPARTITION_BY_COL, [plan_id=117]
         +- FileScan parquet [country#4,amount#5] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/tmp/lab3_orders], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<country:string,amount:double>




In [11]:
# giam partition

spark.conf.set("spark.sql.shuffle.partitions", 50)

df_p = df.repartition("country") \
  .groupBy("country") \
  .sum("amount").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[country#4], functions=[sum(amount#5)])
   +- HashAggregate(keys=[country#4], functions=[partial_sum(amount#5)])
      +- Exchange hashpartitioning(country#4, 50), REPARTITION_BY_COL, [plan_id=132]
         +- FileScan parquet [country#4,amount#5] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/tmp/lab3_orders], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<country:string,amount:double>




LAB 5 là lab quan trọng nhất trong Spark. Nếu LAB 4 là “đốt tiền”, thì LAB 5 là “nổ máy” 💥 (OOM, fail job, incident ban đêm).

Ta làm chậm – rất rõ – từng bước.

---

# 💣 LAB 5 – JOIN, BROADCAST & OOM

> *“90% Spark incident trong bank đến từ JOIN”*

---

## 🎯 Mục tiêu LAB 5

**Sau lab này bạn phải trả lời được:**

*	Vì sao JOIN gây shuffle
*	Vì sao broadcast sai nguy hiểm hơn shuffle
*	**Khi nào Spark chọn:**
-	BroadcastHashJoin
-	SortMergeJoin
-	Vì sao OOM thường đến từ JOIN
-	Cách JOIN bank-grade

---

## 1️⃣ Chuẩn bị data (NHỎ – để hiểu)

Customers (lookup table – nhỏ)

```python
from pyspark.sql import functions as F

customers = spark.createDataFrame([
    ("1", "VN", "LOW"),
    ("2", "SG", "MED"),
    ("3", "TH", "HIGH"),
], ["customer_id", "country", "risk"])

customers.show()
```

Orders (fact table – lớn hơn)

```python
orders = spark.createDataFrame([
    ("o1", "1", 100.0),
    ("o2", "1", 200.0),
    ("o3", "2", 300.0),
    ("o4", "3", 400.0),
], ["order_id", "customer_id", "amount"])

orders.show()
```

---

## 2️⃣ JOIN cơ bản (KHÔNG broadcast)

```python
orders.join(customers, "customer_id").explain()
```

### 🔥 Quan sát Physical Plan

Bạn sẽ thấy:

```code
SortMergeJoin
Exchange hashpartitioning(customer_id, 200)
```
👉 Có Exchange = SHUFFLE

---

## 3️⃣ Vì sao JOIN gây shuffle?

**JOIN cần:**

*	Record cùng customer_id
*	Nhưng data đang ở nhiều executor

👉 Spark phải shuffle cả 2 bảng

📌 JOIN = shuffle × 2 bảng

---

## 4️⃣ Broadcast Join (VŨ KHÍ NGUY HIỂM)

```python
from pyspark.sql.functions import broadcast

orders.join(
    broadcast(customers),
    "customer_id"
).explain()
```

### 🔥 Quan sát plan

```code
BroadcastHashJoin
BroadcastExchange
```

👉 Không còn Exchange trên orders

---

## 5️⃣ Broadcast thực sự là gì?
*	Spark copy bảng nhỏ
*	Gửi sang tất cả executor
*	Mỗi executor JOIN local

📌 Nhanh vì không shuffle fact table

---

6️⃣ Vì sao broadcast có thể gây OOM?

Giả sử:

*	customers = 5GB
*	executor memory = 4GB

👉 Mỗi executor phải load 5GB

💥 JVM chết

💥 Job fail

💥 Incident

---

## 7️⃣ Tắt auto broadcast để quan sát

```python
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

orders.join(customers, "customer_id").explain()
```

👉 Spark bắt buộc dùng SortMergeJoin

📌 Đây là cách debug JOIN strategy

---

## 8️⃣ Khi nào Spark chọn BroadcastHashJoin?

**Spark chọn broadcast khi:**

*	Bảng nhỏ < `spark.sql.autoBroadcastJoinThreshold` (10MB default)
*	Join key đơn giản
*	Không bị hint override

---

## 9️⃣ Khi nào Spark chọn SortMergeJoin?
*	Cả 2 bảng lớn
*	Broadcast bị disable
*	Join key sortable

📌 SortMergeJoin = shuffle nặng nhất

---

## 🔟 So sánh chiến lược JOIN

|**Strategy**|**Shuffle**|**Risk**|
|------------|-----------|--------|
|BroadcastHashJoin|	❌|	💣 OOM|
|SortMergeJoin	|✅	|🐌 Chậm|
|ShuffledHashJoin	|✅|	⚠️ Ít dùng|


---

## 1️⃣1️⃣ JOIN bank-grade (QUY TẮC SỐNG)

### ✅ NÊN
*	Broadcast lookup < 50MB
*	Broadcast dimension table
*	Explain trước khi chạy
*	Cache lookup nếu reuse

### ❌ KHÔNG
*	Broadcast table không rõ size
*	Join nhiều bảng lớn
*	Join trong Gold layer

---

## 1️⃣2️⃣ Câu hỏi sống còn (PHẢI TRẢ LỜI)

**Trả lời bằng ý hiểu của bạn:**

1.	Vì sao JOIN luôn nguy hiểm hơn GROUP BY?
2.	Broadcast nhanh hơn vì sao?
3.	Vì sao broadcast sai gây OOM?
4.	Khi nào nên tắt auto broadcast?
5.	Join trong Gold layer vì sao bị cấm?



In [12]:
from pyspark.sql import functions as F

customers = spark.createDataFrame([
    ("1", "VN", "LOW"),
    ("2", "SG", "MED"),
    ("3", "TH", "HIGH"),
], ["customer_id", "country", "risk"])

customers.show()

+-----------+-------+----+
|customer_id|country|risk|
+-----------+-------+----+
|          1|     VN| LOW|
|          2|     SG| MED|
|          3|     TH|HIGH|
+-----------+-------+----+



In [13]:
orders = spark.createDataFrame([
    ("o1", "1", 100.0),
    ("o2", "1", 200.0),
    ("o3", "2", 300.0),
    ("o4", "3", 400.0),
], ["order_id", "customer_id", "amount"])

orders.show()

+--------+-----------+------+
|order_id|customer_id|amount|
+--------+-----------+------+
|      o1|          1| 100.0|
|      o2|          1| 200.0|
|      o3|          2| 300.0|
|      o4|          3| 400.0|
+--------+-----------+------+



In [14]:
orders.join(customers, "customer_id").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [customer_id#58, order_id#57, amount#59, country#45, risk#46]
   +- SortMergeJoin [customer_id#58], [customer_id#44], Inner
      :- Sort [customer_id#58 ASC NULLS FIRST], false, 0
      :  +- Exchange hashpartitioning(customer_id#58, 50), ENSURE_REQUIREMENTS, [plan_id=185]
      :     +- Filter isnotnull(customer_id#58)
      :        +- Scan ExistingRDD[order_id#57,customer_id#58,amount#59]
      +- Sort [customer_id#44 ASC NULLS FIRST], false, 0
         +- Exchange hashpartitioning(customer_id#44, 50), ENSURE_REQUIREMENTS, [plan_id=186]
            +- Filter isnotnull(customer_id#44)
               +- Scan ExistingRDD[customer_id#44,country#45,risk#46]




In [15]:
from pyspark.sql.functions import broadcast

orders.join(
    broadcast(customers),
    "customer_id"
).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [customer_id#58, order_id#57, amount#59, country#45, risk#46]
   +- BroadcastHashJoin [customer_id#58], [customer_id#44], Inner, BuildRight, false
      :- Filter isnotnull(customer_id#58)
      :  +- Scan ExistingRDD[order_id#57,customer_id#58,amount#59]
      +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, false]),false), [plan_id=215]
         +- Filter isnotnull(customer_id#44)
            +- Scan ExistingRDD[customer_id#44,country#45,risk#46]




In [16]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

orders.join(customers, "customer_id").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [customer_id#58, order_id#57, amount#59, country#45, risk#46]
   +- SortMergeJoin [customer_id#58], [customer_id#44], Inner
      :- Sort [customer_id#58 ASC NULLS FIRST], false, 0
      :  +- Exchange hashpartitioning(customer_id#58, 50), ENSURE_REQUIREMENTS, [plan_id=243]
      :     +- Filter isnotnull(customer_id#58)
      :        +- Scan ExistingRDD[order_id#57,customer_id#58,amount#59]
      +- Sort [customer_id#44 ASC NULLS FIRST], false, 0
         +- Exchange hashpartitioning(customer_id#44, 50), ENSURE_REQUIREMENTS, [plan_id=244]
            +- Filter isnotnull(customer_id#44)
               +- Scan ExistingRDD[customer_id#44,country#45,risk#46]




LAB 6 – đây là nút thắt cuối cùng khiến Spark “đứng hình” dù code đúng & cluster còn trống.

---

# 💥 LAB 6 – DATA SKEW, SALTING & AQE

> *“Job chạy 99% rồi… đứng luôn 40 phút”*

---

## 🎯 Mục tiêu LAB 6

**Sau lab này bạn phải hiểu sâu:**

*	Data skew là gì (bằng MẮT 👀)
*	Vì sao 1 key có thể giết cả cluster
*	Vì sao retry không cứu được skew
*	Salting là gì – dùng khi nào
*	AQE xử lý skew thế nào (Spark 3+ / 4)

---

## 1️⃣ Tạo data có SKEW (RÕ RÀNG)

Orders – 1 customer chiếm 70%

```python
from pyspark.sql import functions as F

orders = (
    spark.range(0, 100_000)
    .select(
        F.col("id").alias("order_id"),
        F.when(F.col("id") < 70_000, F.lit("HOT"))
         .otherwise(F.lit("COLD"))
         .alias("customer_id"),
        (F.rand() * 1000).alias("amount")
    )
)

orders.groupBy("customer_id").count().show()
```

#### 👉 Kết quả:

```code
HOT   ~70,000
COLD  ~30,000
```

📌 Đây là skew

---

## 2️⃣ GroupBy gây thảm họa

```python
orders.groupBy("customer_id").sum("amount").explain()
```

Bạn sẽ thấy:

```code
Exchange hashpartitioning(customer_id, 200)
```

👉 1 partition xử lý 70% dữ liệu

---

## 3️⃣ Điều gì xảy ra trong thực tế?

*	199 task: DONE
*	1 task (HOT key): chạy mãi
*	Stage stuck 99%

📌 Cluster rảnh nhưng job không xong

---

## 4️⃣ Vì sao retry không cứu được skew?

*	Retry = chạy lại cùng logic
*	Skew vẫn tồn tại
*	Task HOT vẫn chết

👉 Retry ≠ Fix design

---

## 5️⃣ SALTING – Cách cổ điển (NHƯNG HIỆU QUẢ)

Thêm salt cho HOT key

```python
from pyspark.sql.functions import concat, lit, rand, floor

orders_salted = orders.withColumn(
    "salted_key",
    F.when(
        F.col("customer_id") == "HOT",
        concat(F.col("customer_id"), lit("_"), floor(rand() * 10))
    ).otherwise(F.col("customer_id"))
)
```

---

GroupBy với salted key

```python
orders_salted.groupBy("salted_key").sum("amount").show()
```

👉 HOT được chia thành:

```code
HOT_0
HOT_1
...
HOT_9
```

📌 Skew được chia nhỏ

---

## 6️⃣ Nhược điểm của salting

* ❌ Code phức tạp
* ❌ Phải “unsalt” lại
* ❌ Không generic
* ❌ Khó maintain

👉 Bank rất ngại salting thủ công

---

## 7️⃣ AQE – Adaptive Query Execution (CỨU TINH)

#### Bật AQE

```pythin
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", "true")
```

---

#### Chạy lại job

```python
orders.groupBy("customer_id").sum("amount").explain()
```
#### 👉 Spark sẽ:
*	Detect skew partition
*	Tự động split
*	Không cần salting thủ công

📌 AQE = salting tự động ở runtime

---

## 8️⃣ Khi nào AQE KHÔNG cứu được?

* ❌ Spark < 3
* ❌ Join logic quá phức tạp
* ❌ Skew cực nặng (1 key = 99%)
* ❌ Memory không đủ

---

## 9️⃣ Bank-grade rules về skew

### ✅ NÊN
*	Detect skew sớm (groupBy count)
*	Bật AQE mặc định
*	Log skew metrics
*	Thiết kế data tránh hot key

### ❌ KHÔNG
*	Retry vô hạn
*	Broadcast table skewed
*	Join skewed key trong Gold

---

## 🔟 CHECKPOINT TƯ DUY (RẤT QUAN TRỌNG)

**Trả lời bằng lời của bạn:**

1.	Vì sao skew làm Spark chậm dù CPU rảnh?
2.	Vì sao 1 task lâu block cả stage?
3.	Salting giải quyết skew bằng cách nào?
4.	AQE tốt hơn salting ở điểm nào?
5.	Khi nào KHÔNG nên dùng salting?


In [17]:
# tao data skew

from pyspark.sql import functions as F

orders = (
    spark.range(0, 100_000)
    .select(
        F.col("id").alias("order_id"),
        F.when(F.col("id") < 70_000, F.lit("HOT"))
         .otherwise(F.lit("COLD"))
         .alias("customer_id"),
        (F.rand() * 1000).alias("amount")
    )
)

orders.groupBy("customer_id").count().show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|        HOT|70000|
|       COLD|30000|
+-----------+-----+



In [18]:
# group by gây thảm hoạ

orders.groupBy("customer_id").sum("amount").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[customer_id#72], functions=[sum(amount#73)])
   +- Exchange hashpartitioning(customer_id#72, 50), ENSURE_REQUIREMENTS, [plan_id=307]
      +- HashAggregate(keys=[customer_id#72], functions=[partial_sum(amount#73)])
         +- Project [CASE WHEN (id#70L < 70000) THEN HOT ELSE COLD END AS customer_id#72, (rand(-4169074559868208654) * 1000.0) AS amount#73]
            +- Range (0, 100000, step=1, splits=2)




In [19]:
# salting

from pyspark.sql.functions import concat, lit, rand, floor

orders_salted = orders.withColumn(
    "salted_key",
    F.when(
        F.col("customer_id") == "HOT",
        concat(F.col("customer_id"), lit("_"), floor(rand() * 10))
    ).otherwise(F.col("customer_id"))
)

In [20]:
# group by with salting

orders_salted.groupBy("salted_key").sum("amount").show()

+----------+--------------------+
|salted_key|         sum(amount)|
+----------+--------------------+
|     HOT_6|  3480921.2529835147|
|     HOT_0|   3549517.763600345|
|     HOT_1|  3481041.7993842703|
|     HOT_4|  3471754.3236366506|
|     HOT_7|  3448158.3647908983|
|     HOT_8|   3496083.587307894|
|     HOT_3|  3530541.7772539984|
|     HOT_2|  3474265.2077110955|
|     HOT_9|    3493761.31312631|
|     HOT_5|  3527731.2368190824|
|      COLD|1.5049008286771532E7|
+----------+--------------------+



In [21]:
# bật AQE

spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", "true")

orders.groupBy("customer_id").sum("amount").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[customer_id#72], functions=[sum(amount#73)])
   +- Exchange hashpartitioning(customer_id#72, 50), ENSURE_REQUIREMENTS, [plan_id=373]
      +- HashAggregate(keys=[customer_id#72], functions=[partial_sum(amount#73)])
         +- Project [CASE WHEN (id#70L < 70000) THEN HOT ELSE COLD END AS customer_id#72, (rand(-4169074559868208654) * 1000.0) AS amount#73]
            +- Range (0, 100000, step=1, splits=2)




# 🧠 RECAP KIẾN TRÚC – SPARK LAB 4 → 5 → 6

(Shuffle – Join – Skew)

---

## 🧩 1️⃣ Big Picture: Spark thực sự chạy như thế nào?

```mermaid
flowchart LR
    A[Source Data] --> B[Transformations]
    B -->|Action| C[DAG]
    C --> D[Stages]
    D --> E[Tasks]
    E --> F[Executors]
```

📌 Mọi thứ chỉ xảy ra khi có ACTION

📌 Performance chết ở Stage boundary (shuffle)

---

## 🔀 2️⃣ LAB 4 – SHUFFLE (KẺ GIẾT SPARK)

Khi nào shuffle xảy ra?

```mermaid
flowchart TD
    A[map / filter] -->|NO shuffle| B[Same partition]
    B --> C[groupBy / join / distinct]
    C -->|SHUFFLE| D[Exchange]
```

**Vì sao shuffle nguy hiểm?**

-	Disk I/O
-	Network transfer
-	Memory pressure
-	Stage mới → block pipeline

> 📌 Mỗi Exchange = rủi ro

---

## 🔗 3️⃣ LAB 5 – JOIN STRATEGIES

Các chiến lược join

```mermaid
flowchart LR
    A[Join] --> B{Table nhỏ?}
    B -->|YES| C[BroadcastHashJoin]
    B -->|NO| D{Sorted?}
    D -->|YES| E[SortMergeJoin]
    D -->|NO| F[ShuffleHashJoin]
```

**Thứ tự ưu tiên (Spark):**

1.	BroadcastHashJoin ⭐
2.	SortMergeJoin
3.	ShuffleHashJoin ❌

> 📌 Join sai = cost + runtime + OOM

---

## 💥 4️⃣ LAB 6 – DATA SKEW (KẺ THÙ THẦM LẶNG)

Skew trông như thế nào?

```mermaid
flowchart LR
    P1[Partition 1] -->|1% data| OK1[Fast]
    P2[Partition 2] -->|1% data| OK2[Fast]
    P3[Partition 3] -->|97% data| SLOW[Very slow 😱]
```

> 📌 1 task chậm → cả stage đứng hình

---

## 🧂 5️⃣ SALTING vs AQE

### Salting (manual)

```mermaid
flowchart LR
    HOT[HOT key] --> S1[HOT_1]
    HOT --> S2[HOT_2]
    HOT --> S3[HOT_3]
```

**✔ Chia tải**
❌ Code phức tạp

❌ Khó maintain

---

### AQE (Adaptive Query Execution)

```mermaid
flowchart LR
    Spark[Runtime] --> Detect[Detect Skew]
    Detect --> Split[Split Partition]
    Split --> Balanced[Balanced Tasks]
```

✔ Tự động

✔ Clean code

✔ Bank thích 😄

❌ Không cứu được design sai nặng

---

## 🏦 6️⃣ BANK-GRADE RULES (CỰC QUAN TRỌNG)

### ❌ KHÔNG LÀM
-	Join nhiều bảng lớn ở Gold
-	Retry vô hạn
-	Broadcast table skew
-	Tin config cứu design

### ✅ PHẢI LÀM
-	Explain plan trước khi chạy
-	Detect skew sớm
-	AQE bật mặc định
-	Job < SLA / 2
-	Idempotent output

---

## 🧠 7️⃣ TƯ DUY KIẾN TRÚC SƯ (ĐINH NÃO)

Spark chậm không phải vì cluster yếu
→ Mà vì data shape & execution plan sai

---

## 🔜 BƯỚC TIẾP THEO – LAB 7 (THỰC CHIẾN TỔNG HỢP)

LAB 7 sẽ làm gì?

✔ Orders lớn (skew)

✔ Customers nhỏ

✔ Join đúng chiến lược

✔ AQE bật

✔ Explain & reasoning như production

✔ Phân tích “nếu job này chạy ở bank thì ổn chưa?”

